In [ ]:
# step 1
import numpy as np


class Variable:
    def __init__(self, data):
        self.data = data


data = np.array(1.0)
x = Variable(data)
print(x.data)

x.data = np.array(2.0)
print(x.data)

# 클래스의 구성요소는 두개다. 1. 메서드(기능) , 2. 속성(데이터)
# 객체를 만드는 순간 2. 속성(데이터)는 상태를 띠는것으로 해석가능하다.
# 클래스(설계도)에 특정 기능,데이터가 들어가 있다면 이로 만든 실체(객체)에도 당연히 동일한 특정 기능 , 데이터가 상태로 변해서 만들어진다!
# 메서드(기능)는 클래스에 한 번만 정의되어, 모든 객체가 공유해서 사용.
# 속성(데이터)은 객체마다 따로 저장되어 **상태(state)**를 형성.
# "클래스에서 정의한 속성(데이터)은 객체가 생성될 때, 해당 객체의 고유한 상태값으로 초기화된다."

1.0
2.0


In [ ]:
# step 2
import numpy as np


class Variable:
    def __init__(self, data):
        self.data = data


class Function:
    def __call__(self, input):
        x = input.data
        y = self.forward(x)
        output = Variable(y)
        return output

    def forward(self, in_data):
        raise NotImplementedError()
    
    # Function 클래스는 추상 클래스(abstract class) 역할을 하며,
# "forward는 각기 다른 연산(예: 제곱, 덧셈 등)에 맞게 자식 클래스에서 구현해라!" 라는 의미입니다.
# def forward(self, in_data):
        #raise NotImplementedError() 가 실행되는 경우들
# 1.Function 클래스를 직접 사용해서 forward()를 호출할 때
# 2.Function 클래스를 상속받았지만 forward()를 재정의하지 않고 호출할 때

class Square(Function):
    def forward(self, x):
        return x ** 2


x = Variable(np.array(10))
f = Square()    # 하는 순간 f 의 __call__ , f 의 forward 가 존재됨!
y = f(x)        # f(x) 실행 시, 파이썬은 f.__call__(x)를 호출하고, 이때 f는 __call__ 메서드의 self가 된다.
print(type(y))  # f 라는 객체의 __call__ 이라는 기능을 호출함! --> self = 현재 사고중인 객체 = f
print(y.data)   

# 파이썬의 기본 생성자는 __init__()을 정의하지 않은 클래스에 자동으로 제공되는 "빈" 초기화 메서드로, 객체 생성만 담당한다.

# 메서드 = 어떤 객체의 기능
# 함수 = 객체를 미포함한 기능
# 속성 = 어떤 객체의 데이터
# 변수 = 객체를 미포함한 데이터

<class '__main__.Variable'>
100


In [ ]:
# step 3
import numpy as np


class Variable:
    def __init__(self, data):
        self.data = data


class Function:
    def __call__(self, input):
        x = input.data
        y = self.forward(x)
        output = Variable(y)
        return output

    def forward(self, x):
        raise NotImplementedError()


class Square(Function):
    def forward(self, x):
        return x ** 2


class Exp(Function):
    def forward(self, x):
        return np.exp(x)


A = Square()
B = Exp()
C = Square()

x = Variable(np.array(0.5))
a = A(x)        # A.__call__(x)
b = B(a)
y = C(b)
print(y.data)

In [ ]:
# step 4

import numpy as np


class Variable:
    def __init__(self, data):
        self.data = data


class Function:
    def __call__(self, input):
        x = input.data
        y = self.forward(x)
        output = Variable(y)
        self.input = input
        self.output = output
        return output

    def forward(self, x):
        raise NotImplementedError()


class Square(Function):
    def forward(self, x):
        return x ** 2


class Exp(Function):
    def forward(self, x):
        return np.exp(x)


def numerical_diff(f, x, eps=1e-4):
    x0 = Variable(x.data - eps)
    x1 = Variable(x.data + eps)
    y0 = f(x0)
    y1 = f(x1)
    return (y1.data - y0.data) / (2 * eps)


f = Square()
x = Variable(np.array(2.0))
dy = numerical_diff(f, x)
print(dy)


def f(x):
    A = Square()
    B = Exp()
    C = Square()
    return C(B(A(x)))


x = Variable(np.array(0.5))
dy = numerical_diff(f, x)
print(dy)

In [ ]:
# step 6

import numpy as np


class Variable:
    def __init__(self, data):
        self.data = data
        self.grad = None


class Function:
    def __call__(self, input):
        x = input.data
        y = self.forward(x)
        output = Variable(y)
        self.input = input
        return output

    def forward(self, x):
        raise NotImplementedError()

    def backward(self, gy):
        raise NotImplementedError()


class Square(Function):
    def forward(self, x):
        self.y = x ** 2     # forward 내부의 현재 목표? x 를 제곱해서 이 데이터를 저장하고 반환하는 것! --> 속성을 사용한 초기화 변수든 일반 지역변수든 둘다 현재 목표를 만족 가능하다. 
                            # 근데 차이는 객체에 속하게 만들것인가 아닌가이다!
        return self.y
    def backward(self, gy):
        x = self.input.data     # self.input이 __call__ 에서와 연동되는게 아니라 그저 "객체에 속하는 성질"을 그대로 살리기 위한 작업(실제 객체를 구체화해서 생각해보면 이해 빨리 됨!) 
        gx = 2 * x * gy     # 그냥 A.q 같은 값이 저장될때 동일한 값을 사용하기 위해 의도한것!
        return gx


class Exp(Function):
    def forward(self, x):
        y = np.exp(x)
        return y

    def backward(self, gy):
        x = self.input.data
        gx = np.exp(x) * gy
        return gx


A = Square()        # A.forward.y
B = Exp()
C = Square()
x = Variable(np.array(0.5))
a = A(x)        # A.__call__(x)
b = B(a)
y = C(b)
print(A.y)
y.grad = np.array(1.0)
b.grad = C.backward(y.grad)
a.grad = B.backward(b.grad)
x.grad = A.backward(a.grad)
print(x.grad)

# 메서드 = 어떤 객체의 기능
# 함수 = 주인 객체를 나타내지 않는 기능
# 속성 = 어떤 객체의 데이터
# 변수 = 주인 객체를 나타내지 않는 데이터 

0.25
3.297442541400256


In [ ]:
# step 7

import numpy as np


class Variable:
    def __init__(self, data):
        self.data = data
        self.grad = None
        self.creator = None

    def set_creator(self, func):
        self.creator = func

    def backward(self):
        f = self.creator  # 1. Get a function
        if f is not None:
            x = f.input  # 2. Get the function's input
            x.grad = f.backward(self.grad)  # 3. Call the function's backward
            x.backward()


class Function:
    def __call__(self, input):
        x = input.data
        y = self.forward(x)
        output = Variable(y)
        output.set_creator(self)  # Set parent(function)
        self.input = input
        self.output = output  # Set output
        return output

    def forward(self, x):
        raise NotImplementedError()

    def backward(self, gy):
        raise NotImplementedError()


class Square(Function):
    def forward(self, x):
        y = x ** 2
        return y

    def backward(self, gy):
        x = self.input.data
        gx = 2 * x * gy
        return gx


class Exp(Function):
    def forward(self, x):
        y = np.exp(x)
        return y

    def backward(self, gy):
        x = self.input.data
        gx = np.exp(x) * gy
        return gx


A = Square()
B = Exp()
C = Square()

x = Variable(np.array(0.5))
a = A(x)
b = B(a)
y = C(b)

# backward
y.grad = np.array(1.0)
y.backward()
print(x.grad)